In [1]:
import os
import pathlib
import sys

default_dir_path = str(pathlib.Path(os.getcwd()).parent.absolute())

sys.path.append(default_dir_path)

from properties import ApplicationProperties

[2021-07-02 07:20:14,536][INFO][properties.py:22] Complete to apply the random seed, RANDOM_SEED : 777


In [2]:
import torch
from torch import optim
from torch.nn import functional as F

from config.config import Config
from dataset.factory import DatasetModule
from domain.base import Hyperparameters
from domain.metadata import Metadata
from logger import logger
from model.factory import ModelModule
from trainer.factory import TrainerModule

In [3]:
mode = "train"
config_file_name = "kaggle_casting_data-cnn_custom-0.yaml"

In [4]:
# Get Parameters
params = Config(file_name=config_file_name).params
logger.info(f"Parameter information :\n{params}")

metadata_params = params.metadata
dataset_params = params.dataset
model_params = params.model
trainer_params = params.trainer

[2021-07-02 07:20:15,047][INFO][<ipython-input-4-98e02a61a8a2>:3] Parameter information :
"dataset":  "hparmas": "key": value
"metadata": "dataset_name":   kaggle_casting_data
"env":            script
"model_name":     cnn_custom
"representation": kaggle_casting_data-cnn_custom
"version":        0
"model":    "hparams": "key": value
"trainer":  "device":           cuda:6
"is_saved":         True
"training_hparams": "key": value


In [5]:
# Metadata Controller
metadata = Metadata(**metadata_params)

In [6]:
# Dataset Controller
dataset_module = DatasetModule(metadata=metadata, **dataset_params)

[2021-07-02 07:20:15,390][INFO][factory.py:81] Data selected : 'kaggle_casting_data'


In [7]:
# Model Controller
model_module = ModelModule(metadata=metadata, **model_params)

[2021-07-02 07:20:15,615][INFO][factory.py:32] Model selected :
'CNNCustom(
  (conv_1): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1))
    (1): LeakyReLU(negative_slope=0.01, inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_2): Sequential(
    (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): LeakyReLU(negative_slope=0.01, inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_3): Sequential(
    (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): LeakyReLU(negative_slope=0.01, inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=19600, out_features=224, bias=True)
    (1): LeakyReLU(negative_slope=0.01, inplace=True)
    (2): Linear(in_features=224, out_features=1, bias=True)
  )
)'


In [8]:
# Trainer Controller
trainer_module = TrainerModule(
    metadata=metadata,
    model_module=model_module,
    dataset_module=dataset_module,
    **trainer_params
)

[2021-07-02 07:20:20,668][INFO][base.py:51] Model set to 'cuda:6'
[2021-07-02 07:20:20,670][INFO][factory.py:50] Trainer selected : 'kaggle_casting_data_cnn_custom_trainer'


In [9]:
trainer_module.trainer.hparams = Hyperparameters(
    optimizer_cls=optim.Adam,
    criterion=F.binary_cross_entropy,
    n_epoch=5,
    lr=1e-3,
    hypothesis_threshold=0.5,
    weight_decay=0,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
)

## Training

In [10]:
result_dict = trainer_module.do(mode="train")

[2021-07-02 07:20:20,702][INFO][factory.py:78] Start to train
  0%|          | 0/5 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
 20%|██        | 1/5 [00:17<01:09, 17.27s/it]

[Epoch - 0]
Train - Accuracy : 0.4331373, Precision : 0.4329647, Recall : 0.994087, F1 : 0.6032081
Val - Accuracy : 0.3664336, Precision : 0.3664336, Recall : 1.0, F1 : 0.5363357
Best Accuracy : 0.36643356643356645 (epoch : 0)
Best F1 : 0.5363357215967247 (epoch : 0)


/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
 40%|████      | 2/5 [00:34<00:51, 17.28s/it]

[Epoch - 1]
Train - Accuracy : 0.4334389, Precision : 0.4334389, Recall : 1.0, F1 : 0.6047539
Val - Accuracy : 0.3664336, Precision : 0.3664336, Recall : 1.0, F1 : 0.5363357
Best Accuracy : 0.36643356643356645 (epoch : 0)
Best F1 : 0.5363357215967247 (epoch : 0)


/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
 60%|██████    | 3/5 [00:51<00:34, 17.32s/it]

[Epoch - 2]
Train - Accuracy : 0.4334389, Precision : 0.4334389, Recall : 1.0, F1 : 0.6047539
Val - Accuracy : 0.3664336, Precision : 0.3664336, Recall : 1.0, F1 : 0.5363357
Best Accuracy : 0.36643356643356645 (epoch : 0)
Best F1 : 0.5363357215967247 (epoch : 0)


/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
 80%|████████  | 4/5 [01:09<00:17, 17.27s/it]

[Epoch - 3]
Train - Accuracy : 0.4334389, Precision : 0.4334389, Recall : 1.0, F1 : 0.6047539
Val - Accuracy : 0.3664336, Precision : 0.3664336, Recall : 1.0, F1 : 0.5363357
Best Accuracy : 0.36643356643356645 (epoch : 0)
Best F1 : 0.5363357215967247 (epoch : 0)


/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
100%|██████████| 5/5 [01:26<00:00, 17.32s/it]
[2021-07-02 07:21:47,405][INFO][base.py:72] Succeed to load best model, device: 'cuda:6'
[2021-07-02 07:21:47,407][INFO][factory.py:86] Completed to train


[Epoch - 4]
Train - Accuracy : 0.4334389, Precision : 0.4334389, Recall : 1.0, F1 : 0.6047539
Val - Accuracy : 0.3664336, Precision : 0.3664336, Recall : 1.0, F1 : 0.5363357
Best Accuracy : 0.36643356643356645 (epoch : 0)
Best F1 : 0.5363357215967247 (epoch : 0)


## Predict

In [11]:
trainer_module.trainer.load_best_model()

[2021-07-02 07:22:06,676][INFO][base.py:72] Succeed to load best model, device: 'cuda:6'


In [12]:
result_dict = trainer_module.do(mode="inference")

[2021-07-02 07:22:09,815][INFO][factory.py:78] Start to inference
/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
[2021-07-02 07:22:11,194][INFO][factory.py:86] Completed to inference


In [13]:
result_dict

{'loss': 61.54891304347826,
 'accuracy': 0.36643356643356645,
 'precision': 0.36643356643356645,
 'recall': 1.0,
 'f1': 0.5363357215967247}